In [1]:
from neo4j import GraphDatabase
import time

In [2]:
# Get Connection to neo4j

uri = 'bolt://localhost:7687'
username = 'neo4j'
password = 'graph'

neo4j_driver = GraphDatabase.driver(uri, auth=(username,password))

In [9]:
# DATA IS LOADED USING neo4j-import


#Netflow = "LOAD CSV WITH HEADERS FROM 'file:///nf_200.csv' AS row MERGE (m:Devices {Name:row.src_device}) MERGE (n:Devices {Name:row.dst_device}) MERGE (m) -[:Netflow { epoch_time:toInteger(row.epoch_time), duration:toInteger(row.duration), protocol:toInteger(row.protocol), src_port:toInteger(row.src_port), dst_port:toInteger(row.dst_port), src_packets:toInteger(row.src_packets), dst_packets:toInteger(row.dst_packets), src_bytes:toInteger(row.src_bytes), dst_bytes:toInteger(row.dst_bytes) }]-> (n)"

#start = time.time()
#with neo4j_driver.session() as neo4j_Session:
#    neo4j_Session.run(Netflow)


In [4]:
# DATA IS LOADED USING neo4j-import


#HostEvents = "LOAD CSV WITH HEADERS FROM 'file:///1v_200.csv' AS row MERGE (m:Devices {Name:row.log_host}) MERGE (m) -[:HostEvents { epoch_time:toInteger(row.epoch_time),event_id:toInteger(row.event_id),user_name:row.user_name, domain_name:row.domain_name, logon_id: toInteger(row.logon_id), process_name:row.process_name, process_id:toInteger(row.process_id), parent_process_name:row.parent_process_name, parent_process_id:toInteger(row.parent_process_id) }]-> (m)" 
                         
#start = time.time()
#with neo4j_driver.session() as neo4j_Session:
#    neo4j_Session.run(HostEvents)

In [3]:
SSHFlow_Query = """MATCH (v0:Devices)-[edge:Netflow]->(v1:Devices) 
WHERE edge.dst_port=22 
CREATE (v0)-[e:SSHFlow 
             {epoch_time : toInteger(edge.epoch_time), duration : toInteger(edge.duration), 
              protocol : toInteger(edge.protocol), src_port : toInteger(edge.src_port), 
              dst_port : toInteger(edge.dst_port)}]->(v1) RETURN count(*)"""

with neo4j_driver.session() as neo4j_Session:
    result1 = neo4j_Session.run(SSHFlow_Query)

print(result1.data())

[{'count(*)': 435663}]


In [ ]:
Final_Query = """MATCH (n1:Devices)-[r1:SSHFlow]->(n2:Devices) 
MATCH (n1)-[r2:SSHFlow]->(n2) 
MATCH (n2)-[rb:HostEvents]->(n2) 
WHERE rb.event_id=1100 
AND n1 <> n2 
AND r1 <> r2 
AND r1.epoch_time <= r2.epoch_time 
AND r2.epoch_time - r1.epoch_time < 60 
AND r2.epoch_time <= rb.epoch_time 
AND rb.epoch_time - r2.epoch_time < 180 
RETURN n1,n2,r1.epoch_time,r2.epoch_time,rb.epoch_time limit 1000"""


with neo4j_driver.session() as neo4j_Session:
    result2 = neo4j_Session.run(Final_Query)

print(result2.data())